In [2]:
import pandas as pd
import scipy.stats as stats
import os

Concatenate our files. Reminder: we have a file per year, we all of 'Informatique' in it.

In [3]:
# requires os
def concatFiles(direc, fileType):
    files = os.listdir(direc)
    if '.DS_Store' in files:
        files.remove('.DS_Store')
    print(files)
    if fileType == 'csv':
        r = pd.read_csv
    for idx, file in enumerate(files):
        file = direc+file
        if idx == 0:
            df = r(file, header=0)
            print(file)
        else:
            new = r(file, header=0)
            df = pd.concat([df, new], axis=0)
            print(file)
    return df

In [4]:
info = concatFiles('data/', 'csv')

['informatique_2011_2012.csv', 'informatique_2014_2015.csv', 'informatique_2012_2013.csv', 'informatique_2008_2009.csv', 'informatique_2007_2008.csv', 'informatique_2016_2017.csv', 'informatique_2015_2016.csv', 'informatique_2010_2011.csv', 'informatique_2013_2014.csv', 'informatique_2009_2010.csv']
data/informatique_2011_2012.csv
data/informatique_2014_2015.csv
data/informatique_2012_2013.csv
data/informatique_2008_2009.csv
data/informatique_2007_2008.csv
data/informatique_2016_2017.csv
data/informatique_2015_2016.csv
data/informatique_2010_2011.csv
data/informatique_2013_2014.csv
data/informatique_2009_2010.csv


We want to take only the data for the Master students

In [5]:
def locator(s):
    return s.find('Master') != -1

In [6]:
ma = info.loc[info['Semester'].apply(locator)]
ma.head()

,Civilité,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,No Sciper,Subject,Period,Semester
574,Monsieur,Arnfred Jonas,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,184772,Informatique,2011-2012,Master semestre 1
575,Monsieur,Asgari Ehsaneddin,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,211754,Informatique,2011-2012,Master semestre 1
576,Monsieur,Baeriswyl Jonathan,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,179406,Informatique,2011-2012,Master semestre 1
577,Madame,Bai Yi,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,209850,Informatique,2011-2012,Master semestre 1
578,Monsieur,Barroco Michael,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,179428,Informatique,2011-2012,Master semestre 1


We define a function to locate a specific sciper #. They are unique.

In [7]:
def locSciper(df, sciper):
    return df.loc[df['No Sciper'] == sciper]

In [8]:
def getGender(df):
    """ Take a DataFrame and checks the gender
    
    Args:
        df (DataFrame): a DataFrame, typically a .loc on a specific student
        
    Returns:
        bool: True if student is a woman, False otherwise
    """
    try:
        if df.isin(['Madame'])['Civilité'].value_counts()[True] > 0:
            return 1
    except KeyError:
        return 0

Define a function that will return first and last semester depending on different possibilities

In [13]:
def isOneTwo(df):
    """ Take a DataFrame and check that there is an entry for both BA1 and BA6
    
    Args:
        df (DataFrame): a DataFrame, typically a .loc on a specific student
        
    Returns:
        bool: True if it finds both BA1 and BA6, False otherwise
    """
    try:
        one = df.isin(['Master semestre 1'])['Semester'].value_counts()[True] > 0
    except KeyError:
        one = 0
    try:
        two = df.isin(['Master semestre 2'])['Semester'].value_counts()[True] > 0
    except KeyError:
        two = 0
    return (one and two)

Loop through all the students


In [14]:
dico = {'sciper': [], 'gender': [], 'length': []}
for row in ma.iterrows():
    o = row[1]
    sciper = o['No Sciper']
    # Making sure we consider unique sciper #s
    if not sciper in dico['sciper']:
        df = locSciper(ma, sciper)
        if isOneTwo(df):        
            dico['sciper'].append(sciper)
            dico['gender'].append(getGender(df))
            # Calculating length of stay by nbr of rows
            dico['length'].append(len(df))

In [15]:
data = pd.DataFrame(dico)

In [16]:
data['length'].describe()

count    764.000000
mean       3.391361
std        1.023948
min        2.000000
25%        3.000000
50%        3.000000
75%        4.000000
max        8.000000
Name: length, dtype: float64

In [17]:
men = data.loc[data.gender == 0]

In [18]:
men.length.mean()

3.3891402714932126

In [19]:
women = data.loc[data.gender == 1]

In [20]:
women.length.mean()

3.405940594059406

In [21]:
stats.ttest_ind(a=women.length, b=men.length, )

Ttest_indResult(statistic=0.15350842579835766, pvalue=0.87803798898545327)